# Projeto 1 - Ciência dos Dados

Nome: Guilherme Galvão Guedes

Nome: _____

Nome: _____

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [3]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\guilh\OneDrive\Documentos\GitHub\CDadosP1


Carregando a base de dados com os tweets classificados manualmente:

In [4]:
train = pd.read_excel('dados_treino.xlsx')
train.head(5)

,Mensagem,target treino,Legenda
0,Achei muito merchandising e pouco conteúdo prá...,1.0,1 = critica construtiva ao livro
1,Não é o primeiro livro da Record que oxida em ...,1.0,2 = critica construtiva a edição/pltaforma
2,"Estava com saudades de todos os personagens, m...",1.0,0 = nenhum dos dois
3,"Perdeu toda credibilidade, se realmente fosse ...",1.0,NaN
4,Ainda vou ler.,0.0,NaN


In [5]:
test = pd.read_excel('dados_teste 1.xlsx')
test.head(5)

,Mensagem,target:,Legenda
0,"Não sou de desistir de livro, mas neste não te...",1,1 = critica construtiva ao livro
1,Por ser produto não do meu interesse e recebi...,0,2 = critica construtiva a edição/plataforma
2,"Livro de agradável leitura, bastante esclarece...",0,0 = nenhum dos dois
3,No dia que era para meu livro chegar recebo um...,2,NaN
4,Como pode uma editora aproveitar o sucesso do ...,2,NaN


___
## Classificador automático


Faça aqui uma descrição do seu assunto e o que considerou como relevante ou não relevante na classificação dos tweets (Target).

Nosso projeto considera as seguintes classificações: 1 = Crítica construtiva ao livro, 2 = Crítica construtiva à edição/plataforma e 0 = Nenhum dos dois. Com isso planejamos criar um sistema de identificação de reviews novas, baseando-se em probabilidades encontradas a partir da nossa base de dados.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas categorias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por categoria (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**